In [13]:
from dotenv import load_dotenv

load_dotenv()

True

# Rephrase Question

In [1]:
SYSTEM_PROMPT = """
<PERSONA>
Eres un asistente de contact center
</PERSONA>

<TASK>
Tu tarea es refrasear la solicitud del usuario para genera una solicitud refraseada.

- Puedes corregir los errores gramaticales
- Puedes mejorar la semántica y orden léxico de la palabras para un mejor entendimiento
</TASK>
"""

USER_PROMPT = """{user_request}"""



In [8]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate([
    SystemMessage(content=SYSTEM_PROMPT),
    ("user", USER_PROMPT)
])

In [10]:
prompt_template.invoke({"user_request": "hola"})

ChatPromptValue(messages=[SystemMessage(content='\n<PERSONA>\nEres un asistente de contact center\n</PERSONA>\n\n<TASK>\nTu tarea es refrasear la solicitud del usuario para genera una solicitud refraseada.\n\n- Puedes corregir los errores gramaticales\n- Puedes mejorar la semántica y orden léxico de la palabras para un mejor entendimiento\n</TASK>\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='hola', additional_kwargs={}, response_metadata={})])

# Generate LLM

In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
)

In [15]:
chain = prompt_template | llm

In [16]:
chain.invoke({"user_request": "hola tengo probleas con mi telefono celular, La cuenta vino muy alta"})

AIMessage(content='Hola, tengo problemas con mi teléfono celular. La factura llegó muy alta.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 98, 'total_tokens': 114, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_reason': 'stop', 'logprobs': None}, id='run-dcce00ef-b24d-41a0-b377-c74e2af5f22d-0', usage_metadata={'input_tokens': 98, 'output_tokens': 16, 'total_tokens': 114, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# Introducing Runnable Lambdas

In [17]:
from langchain_core.messages import AIMessage

def convert_message_to_text(message: AIMessage) -> str:
    return message.content

In [18]:
from langchain_core.runnables import RunnableLambda

chain_text = prompt_template | llm | RunnableLambda(convert_message_to_text)

In [20]:
chain_text.invoke({"user_request": "hola tengo probleas con mi telefono celular, La cuenta vino muy alta"})

'Hola, tengo problemas con mi teléfono celular. La factura llegó muy alta.'

# Let's dive into Runnables

In [31]:
from operator import itemgetter
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel({
    "user_request": itemgetter("user_request"),
    "user_request_rephrased": prompt_template | llm | RunnableLambda(convert_message_to_text)
})

In [32]:
chain.invoke({"user_request": "hola tengo probleas con mi telefono celular, La cuenta vino muy alta"})

{'user_request': 'hola tengo probleas con mi telefono celular, La cuenta vino muy alta',
 'user_request_rephrased': 'Hola, tengo problemas con mi teléfono celular. La factura llegó muy alta.'}